## Create Postgres schemas and migrate data from Big Query

In [1]:
import os

import google
from google.cloud import bigquery
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.schema import CreateSchema

from dbcp.metadata.data_mart import counties_proposed_clean_projects, schema as data_mart_schema
from dbcp.metadata.private_data_warehouse import (
    fyi_projects,
    fyi_locations,
    fyi_resource_capacity,
    schema as data_warehouse_schema,
)

### Initialize Postgres connection

In [4]:
url = "os.environ['POSTGRES_URL']"
engine = create_engine(url)

### Create Tables

In [ ]:
counties_proposed_clean_projects.create(engine)
fyi_projects.create(engine)
fyi_locations.create(engine)
fyi_resource_capacity.create(engine)

### Migrate Data

In [ ]:
credentials, project_id = google.auth.default()
client = bigquery.Client(credentials=credentials, project=project_id)

def _get_table(table_name, schema) -> pd.DataFrame:
    return client.query(f"SELECT * FROM `dbcp-dev-350818.{schema}.{table_name}`;").to_dataframe()

# Migrate data warehouse tables
for table_name in ["fyi_projects", "fyi_locations", "fyi_resource_capacity"]:
    df = _get_table(table_name, "private_data_warehouse_dev")
    #df.to_sql(table_name, con=engine, schema=data_warehouse_schema)

# Migrate data mart table
df = _get_table("counties_proposed_clean_projects", "data_mart_dev")
#df.to_sql("fyi_counties_proposed_clean_projects", con=engine, schema=data_mart_schema)